# Spotify API Album Search

This notebook demonstrates how to search for albums on Spotify using the `spotifysearch` library, which simplifies the API interaction and handles authentication via environment variables.

## Search Format

The Spotify API allows searching by various fields:
- `album`: Filter albums or tracks by album name
- `artist`: Filter by artist name
- `track`: Filter by track name
- `year`: Filter by release year (can be a range like "1995-2000")
- `genre`: Filter by genre
- `tag:new`: Filter for albums released in the last two weeks
- `tag:hipster`: Filter for albums in the lowest 10% popularity

For our purpose of searching by artist and album name, we'll use the spotifysearch library which simplifies these queries.

## Setup Required

Before using this notebook, make sure you have set the following environment variables:
- `SPOTIFY_CLIENT_ID`: Your Spotify API client ID
- `SPOTIFY_CLIENT_SECRET`: Your Spotify API client secret

You can get these by creating an app in the [Spotify Developer Dashboard](https://developer.spotify.com/dashboard/applications).

In [1]:
# Define the artist and album to search for
singer = "Taylor Swift"
album = "1989"

In [2]:
# Using the spotifysearch library to search for albums
import os
from spotifysearch.client import Client

# Initialize the Spotify API client
# Get credentials from environment variables for security
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

# Create client instance with credentials
myclient = Client(client_id, client_secret)

# Create search query combining album and artist
search_query = f"{album} {singer}"

# Search using the client
results = myclient.search(search_query)

# Get albums from search results
albums = results.get_albums()

# Print the results in a readable format
if albums:
    print(f"Found {len(albums)} album results")
    
    # Display the first few results
    for i, album_item in enumerate(albums[:3]):
        print(f"\nResult {i+1}:")
        print(f"Album name: {album_item.name}")
        print(f"Artist: {album_item.artists[0].name if album_item.artists else 'Unknown'}")
        print(f"Release date: {album_item.release_date}")
        print(f"Spotify URL: {album_item.url}")
        print(f"Album ID: {album_item.id}")
else:
    print("No results found or API error occurred")
    
# Check if environment variables are set correctly
if not client_id or not client_secret:
    print("\nWARNING: SPOTIFY_CLIENT_ID or SPOTIFY_CLIENT_SECRET environment variables are not set.")
    print("Set these environment variables before running this notebook:")
    print("export SPOTIFY_CLIENT_ID='your_client_id'")
    print("export SPOTIFY_CLIENT_SECRET='your_client_secret'")


No results found or API error occurred


In [3]:
# Extract and display album details in a more structured way

# This cell assumes you've run the previous cell and have results

import pandas as pd

def extract_album_data(album_objects):
    """Extract relevant information from spotifysearch library album objects"""
    albums_data = []
    
    if not album_objects:
        return pd.DataFrame()
    
    for album_item in album_objects:
        album_data = {
            'Album Name': album_item.name,
            'Artist': ', '.join([artist.name for artist in album_item.artists]),
            'Release Date': album_item.release_date,
            'Album Type': album_item.album_type,
            'Spotify URL': album_item.url,
            'Album ID': album_item.id
        }
        
        # Add image URL if available
        if hasattr(album_item, 'images') and album_item.images:
            album_data['Image URL'] = album_item.images[0]['url'] if isinstance(album_item.images[0], dict) else None
        else:
            album_data['Image URL'] = None
            
        albums_data.append(album_data)
    
    return pd.DataFrame(albums_data)

# Create a DataFrame from the results
if 'albums' in locals() and albums:
    df_albums = extract_album_data(albums)
    display(df_albums)
    
    print(f"\nFound {len(df_albums)} albums matching '{album}' by '{singer}'")
else:
    print("No results available. Run the previous cell first.")


No results available. Run the previous cell first.


In [4]:
# Search for tracks in a specific album
# This assumes you've found albums from the previous cells

if 'albums' in locals() and albums and len(albums) > 0:
    # Get the first album from results
    selected_album = albums[0]
    
    print(f"Searching for tracks in album: {selected_album.name} (ID: {selected_album.id})")
    
    # Get tracks for the album - according to spotifysearch documentation
    # We need to create a new request to get the tracks
    if 'myclient' in locals():
        # Get the album with tracks
        album_with_tracks = myclient.get_album(selected_album.id)
        tracks = album_with_tracks.tracks if hasattr(album_with_tracks, 'tracks') else []
        
        if tracks:
            print(f"\nFound {len(tracks)} tracks in the album:")
            
            # Create a DataFrame of tracks
            track_data = []
            for track in tracks:
                track_info = {
                    'Track Number': track.track_number if hasattr(track, 'track_number') else None,
                    'Track Name': track.name,
                    'Duration (ms)': track.duration_ms if hasattr(track, 'duration_ms') else None,
                    'Explicit': track.explicit if hasattr(track, 'explicit') else None,
                    'Preview URL': track.preview_url if hasattr(track, 'preview_url') else None,
                    'Spotify URL': track.url
                }
                track_data.append(track_info)
            
            # Display tracks in a DataFrame
            df_tracks = pd.DataFrame(track_data)
            if 'Track Number' in df_tracks.columns:
                df_tracks = df_tracks.sort_values('Track Number')
            display(df_tracks)
        else:
            print("No tracks found for this album.")
    else:
        print("Client not available. Run the previous cells first.")
else:
    print("No album search results available. Run the previous cells first.")


No album search results available. Run the previous cells first.


In [5]:
# Helper cell to set environment variables if needed
# Uncomment and run this cell if you need to set environment variables in the notebook

'''
import os

# Replace with your actual Spotify API credentials
os.environ["SPOTIFY_CLIENT_ID"] = "your_client_id_here"
os.environ["SPOTIFY_CLIENT_SECRET"] = "your_client_secret_here"

print("Environment variables set. Re-run the search cells to use these credentials.")
'''

# NOTE: It's generally better to set these variables in your shell or .env file
# rather than hardcoding them in the notebook for security reasons
#
# In your terminal:
# export SPOTIFY_CLIENT_ID='your_client_id'
# export SPOTIFY_CLIENT_SECRET='your_client_secret'
#
# Or create a .env file and use a library like python-dotenv to load it

'\nimport os\n\n# Replace with your actual Spotify API credentials\nos.environ["SPOTIFY_CLIENT_ID"] = "your_client_id_here"\nos.environ["SPOTIFY_CLIENT_SECRET"] = "your_client_secret_here"\n\nprint("Environment variables set. Re-run the search cells to use these credentials.")\n'

In [6]:
# Example: Searching for tracks directly
# This shows how to search for tracks instead of albums

if 'myclient' in locals():
    # Define track search query
    track_query = "Love Story Taylor Swift"
    
    print(f"Searching for tracks with query: '{track_query}'")
    
    # Search for tracks
    track_results = myclient.search(track_query)
    tracks = track_results.get_tracks()
    
    if tracks:
        print(f"\nFound {len(tracks)} tracks:")
        
        # Display the first few results
        for i, track in enumerate(tracks[:5]):
            print(f"\nTrack {i+1}:")
            print(f"Name: {track.name}")
            print(f"Artist: {track.artists[0].name if track.artists else 'Unknown'}")
            print(f"Album: {track.album.name if hasattr(track, 'album') else 'Unknown'}")
            print(f"Spotify URL: {track.url}")
            
        # Create a DataFrame for easier viewing
        track_list = [{
            'Track Name': t.name,
            'Artist': ', '.join([a.name for a in t.artists]) if t.artists else 'Unknown',
            'Album': t.album.name if hasattr(t, 'album') and t.album else 'Unknown',
            'Duration (ms)': t.duration_ms if hasattr(t, 'duration_ms') else None,
            'Spotify URL': t.url
        } for t in tracks[:10]]
        
        df_track_search = pd.DataFrame(track_list)
        display(df_track_search)
    else:
        print("No tracks found.")
else:
    print("Client not available. Run the previous cells first.")


Searching for tracks with query: 'Love Story Taylor Swift'

Found 20 tracks:

Track 1:
Name: Love Story
Artist: Taylor Swift
Album: Fearless (Big Machine Radio Release Special)
Spotify URL: https://open.spotify.com/track/1D4PL9B8gOg78jiHg3FvBb

Track 2:
Name: cupid's arrows
Artist: phendste
Album: from the heart
Spotify URL: https://open.spotify.com/track/3ScLA5qriLhyLoV5bH8kP9

Track 3:
Name: Let go
Artist: Nancy Kwai
Album: Let go
Spotify URL: https://open.spotify.com/track/0VKdwugDTY088b1J84QH3e

Track 4:
Name: Love Story (Taylor’s Version)
Artist: Taylor Swift
Album: Love Story (Taylor’s Version)
Spotify URL: https://open.spotify.com/track/3CeCwYWvdfXbZLXFhBrbnf

Track 5:
Name: cupid's arrows
Artist: phendste
Album: cupid's arrows
Spotify URL: https://open.spotify.com/track/1FvT8gJZOfSPWMAihH8iUp

Found 20 tracks:

Track 1:
Name: Love Story
Artist: Taylor Swift
Album: Fearless (Big Machine Radio Release Special)
Spotify URL: https://open.spotify.com/track/1D4PL9B8gOg78jiHg3FvBb

Tr

,Track Name,Artist,Album,Duration (ms),Spotify URL
0,Love Story,Taylor Swift,Fearless (Big Machine Radio Release Special),236266,https://open.spotify.com/track/1D4PL9B8gOg78ji...
1,cupid's arrows,phendste,from the heart,181747,https://open.spotify.com/track/3ScLA5qriLhyLoV...
2,Let go,Nancy Kwai,Let go,230581,https://open.spotify.com/track/0VKdwugDTY088b1...
3,Love Story (Taylor’s Version),Taylor Swift,Love Story (Taylor’s Version),235766,https://open.spotify.com/track/3CeCwYWvdfXbZLX...
4,cupid's arrows,phendste,cupid's arrows,181747,https://open.spotify.com/track/1FvT8gJZOfSPWMA...
5,ON,Anson Lo 盧瀚霆,ON,200027,https://open.spotify.com/track/2Yccw4AWlnf8BMc...
6,Love Story (Taylor’s Version),Taylor Swift,Fearless (Taylor's Version),235766,https://open.spotify.com/track/6YvqWjhGD8mB5QX...
7,Running,"T. Smozz, Valerù",Running,196740,https://open.spotify.com/track/7K1grGAoB72kkW7...
8,would you be mine?,Tyson Yoshi,would you be mine?,190432,https://open.spotify.com/track/4JXFK04oMgM5HPK...
9,Love Story,Taylor Swift,Fearless (Platinum Edition),235280,https://open.spotify.com/track/5hnJCRSo4pxRsaE...
